In [1]:
from cylinder_fdm_3d import CylinderFDM
from icecream import ic
import numpy as np
from scipy.sparse.linalg import eigsh, LinearOperator, cg
from time import time
from matplotlib import pyplot as plt
from fft_tdse.simulator import LaserPulse
from erfgau import ErfgauPotential




In [4]:

n_r = 20
n_z = 20
n_m = 8
solver = CylinderFDM(r_max = 10, z_max = 10, n_r = n_r, n_z = n_z , n_m = n_m)

tt, rr, zz = solver.get_trz_meshgrid()
#rr, zz = np.meshgrid(solver.r_inner, solver.z_inner, indexing='ij')    
ic(tt.shape, rr.shape, zz.shape)
xx = rr*np.cos(tt)
yy = rr*np.sin(tt)

# x0 = 0
# y0 = 0
# alpha = x0 - 1j*y0
# z0 = 0.0
# V_m = []
# # V_m.append(-alpha*0.5*rr)
# # V_m.append(0.5*(zz-z0)**2 + 0.5*rr**2 + 0.5*np.abs(alpha)**2)
# # V_m.append(-alpha.conjugate()*0.5*rr)
# erfgau = ErfgauPotential(mu=1.0)
# V_m.append(erfgau.potential_radial((rr*rr + zz*zz)**.5))

erfgau = ErfgauPotential(mu=1.0)
#V = erfgau.potential_radial((rr*rr + zz*zz)**.5)
V =  0.5*((xx-0.0)**2 + (yy+0.)**2 + zz**2)

ic(np.linalg.norm(V))
# V_m = []
# V_m.append(0.5*(rr**2 + zz**2))
#solver.set_realspace_potential(V, rotation_symmetric=False)
solver.set_realspace_potential(V[0], rotation_symmetric=True)
#solver.set_td_potential(D_m)

N1 = np.linalg.norm(solver.V_m[0]) 
N2 = np.linalg.norm(solver.V[0,:,:])
ic(N1, N2)


ic(solver.V_m[0][0,0], solver.V[0,0,0])

ic| self.m_i: array([ 0.,  1.,  2.,  3., -4., -3., -2., -1.])
ic| cylinder_fdm_3d.py:162 in __init__() at 06:50:54.234
ic| self.n_r: 20, self.n_z: 20, self.n_m: 8
ic| self.r_max: 10, self.z_max: 10
ic| self.n_dof: 3200
ic| tt.shape: (8, 20, 20)
    rr.shape: (8, 20, 20)
    zz.shape: (8, 20, 20)
ic| np.linalg.norm(V): 2088.513912956158
ic| m: 0.0, N: 738.4011752268759
ic| m: 1.0, N: 1.6014703311779717e-14
ic| m: 2.0, N: 1.7484206229301394e-14
ic| m: 3.0, N: 1.6014703311779717e-14
ic| m: -4.0, N: 3.3591998810548666e-14
ic| m: -3.0, N: 1.6014703311779717e-14
ic| m: -2.0, N: 1.7484206229301394e-14
ic| m: -1.0, N: 1.6014703311779717e-14
ic| m_max: 0
ic| m: 0.0
ic| m_list[i]: 0.0, np.linalg.norm(V_m[i]): 738.4011752268759
ic| N1: 738.4011752268759, N2: 738.4011752268759
ic| solver.V_m[0][0,0]: (41.04308390022676+0j)
    solver.V[0,0,0]: 41.04308390022676


((41.04308390022676+0j), 41.04308390022676)

In [5]:

ic('Computing sparse matrix')

start = time()
H_mat_sparse = solver.get_sparse_matrix_fast()
ic(hasattr(solver, 'H_kin'))
ic(hasattr(solver, 'H_pot'))
ic(hasattr(solver, 'H_pot_td'))

time_taken_sparse = time() - start
ic(time_taken_sparse)

H_mat_sparse_slow = solver.get_sparse_matrix()
#time_taken_sparse_slow = time() - start
# ic(time_taken_sparse_slow)

H = H_mat_sparse.todense()
H_slow = H_mat_sparse_slow.todense()

ic(np.linalg.norm(H - H_slow))


ic| 'Computing sparse matrix'
ic| V_m_max: 0
ic| m: 0, m_ind: 0
ic| diagonals.shape: (3,)
ic| data.shape: (3, 400)
    diagonals.shape: (3,)
    diagonals: array([-3200,     0,  3200])
ic| 'Setting total Hamiltonian as self.H_tot'
ic| hasattr(solver, 'H_kin'): True
ic| hasattr(solver, 'H_pot'): True
ic| hasattr(solver, 'H_pot_td'): False
ic| time_taken_sparse: 0.04837989807128906
ic| self.n_dof: 3200
ic| np.linalg.norm(H - H_slow): 1953.6258774481405


1953.6258774481405

In [4]:
E, U = eigsh(H_mat_sparse, k=10, sigma=-0.6)
ic(np.sort(E))
E, U = eigsh(H_mat_sparse_slow, k=10, sigma=-0.6)
ic(np.sort(E))

ic| np.sort(E): array([1.50526377, 2.32424682, 2.33094513, 2.3567457 , 3.17572875,
                       3.18242706, 3.18554129, 3.42153894, 3.42504585, 3.43267545])
ic| np.sort(E): array([1.49282207, 2.3242479 , 2.33727553, 2.344304  , 3.17309958,
                       3.17572983, 3.18875745, 3.41444916, 3.42023375, 3.4254997 ])


array([1.49282207, 2.3242479 , 2.33727553, 2.344304  , 3.17309958,
       3.17572983, 3.18875745, 3.41444916, 3.42023375, 3.4254997 ])

In [9]:
E

array([-0.49326609, -0.11682082, -0.12225825, -0.12237373, -0.12237373])

In [ ]:
psi = np.zeros(solver.shape, dtype=np.complex128)
psi[n_m,...] = rr**.5 * np.exp(-0.5*(rr**2 + zz**2))
psi[n_m,...] = rr**.5 * np.exp(-(rr**2 + zz**2)**.5)
psi = psi.flatten()
psi /= np.linalg.norm(psi)
E = np.dot(psi.conjugate(), H_mat_sparse.dot(psi))
ic(E)

In [4]:
laser = LaserPulse(t0=0, T=100, E0=0.1, omega=0.057)
dt = 0.01
solver.setup_splitting_scheme(dt, td_potential=True, envelope = laser)

/Users/simenkva/anaconda3/lib/python3.10/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:395: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)
/Users/simenkva/anaconda3/lib/python3.10/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:285: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '
ic| 'exp_T_z', time()-start: 0.056507110595703125
ic| 'exp_T_r', time()-start: 0.8096699714660645


KeyboardInterrupt: 

In [5]:
laser = LaserPulse(t0=0, T=100, E0=0.1, omega=0.057)


t_range = np.linspace(0, 100, 1001)
dt = t_range[1] - t_range[0]

solver.setup_propagate(dt=dt, td_potential=True, envelope=laser)

for i in range(len(t_range)):
    t = t_range[i]
    start = time()
    #psi = solver.propagate_exp(psi, t=t)
    psi = solver.propagate_crank_nicolson(psi, t=t)
    #psi /= np.linalg.norm(psi)
    
    E = np.dot(psi.conjugate(), H_mat_sparse.dot(psi)).real
    ic(t, E)

ic| t: 0.0, E: -0.49338710216907994
ic| t: 0.1, E: -0.4933871021692946
ic| t: 0.2, E: -0.49338710216922577


KeyboardInterrupt: 

In [7]:
A = LinearOperator((solver.n_dof, solver.n_dof), matvec=lambda x: solver.apply_hamiltonian(x.reshape(solver.shape)).flatten())

In [12]:
from scipy.sparse.linalg import expm_multiply
psi = expm_multiply(-1j*0.01*H_mat_sparse, U[:,0])

In [27]:
class ExampleLinOp(LinearOperator):
    def __init__(self, solver, dt):
        self.solver = solver
        self.dt = dt
        self.trace = -1j*dt*8094430.916122656
        super().__init__(dtype=complex, shape=(solver.n_dof, solver.n_dof))

    def _matvec(self, v):
        
        return -1j*self.dt*self.solver.apply_hamiltonian(v.reshape(self.solver.shape)).flatten()
    
    def _rmatvec(self, v):
        # operator defined to be Hermitian
        return -self._matvec(v)
    
    
from scipy.sparse.linalg import expm_multiply

psi = U[:,0]

dt = 0.01 

A = ExampleLinOp(solver, dt)

psi2 = expm_multiply(A, psi)

test = psi2/psi

ic(np.average(np.abs(test)))
ic(np.std(np.abs(test)))


/var/folders/qc/b2y8zg713_x554g8ykkcbcbh0000gp/T/ipykernel_12962/1221239713.py:25: UserWarning: Trace of LinearOperator not available, it will be estimated. Provide `traceA` to ensure performance.
  psi2 = expm_multiply(A, psi)
ic| np.average(np.abs(test)): 1.1821938933356382
ic| np.std(np.abs(test)): 1.8926329429674618


1.8926329429674618

In [21]:
H_mat_sparse.trace()

(8094430.916122656+0j)

In [28]:
A = LinearOperator((solver.n_dof, solver.n_dof), matvec=lambda x: solver.apply_hamiltonian(x.reshape(solver.shape)).flatten(), rmatvec=lambda x: solver.apply_hamiltonian(x.reshape(solver.shape)).flatten())

In [9]:
from scipy.sparse.linalg import spsolve
help(spsolve)

Help on function spsolve in module scipy.sparse.linalg._dsolve.linsolve:

spsolve(A, b, permc_spec=None, use_umfpack=True)
    Solve the sparse linear system Ax=b, where b may be a vector or a matrix.
    
    Parameters
    ----------
    A : ndarray or sparse matrix
        The square matrix A will be converted into CSC or CSR form
    b : ndarray or sparse matrix
        The matrix or vector representing the right hand side of the equation.
        If a vector, b.shape must be (n,) or (n, 1).
    permc_spec : str, optional
        How to permute the columns of the matrix for sparsity preservation.
        (default: 'COLAMD')
    
        - ``NATURAL``: natural ordering.
        - ``MMD_ATA``: minimum degree ordering on the structure of A^T A.
        - ``MMD_AT_PLUS_A``: minimum degree ordering on the structure of A^T+A.
        - ``COLAMD``: approximate minimum degree column ordering [1]_, [2]_.
    
    use_umfpack : bool, optional
        if True (default) then use UMFPACK for th

In [7]:
ic(type(solver.H_pot))

ic| type(solver.H_pot): <class 'scipy.sparse._dia.dia_matrix'>


scipy.sparse._dia.dia_matrix

In [13]:
978.4300278041296/345.92725378842124

2.8284271247461894

In [4]:
help(np.tile)

Help on _ArrayFunctionDispatcher in module numpy:

tile(A, reps)
    Construct an array by repeating A the number of times given by reps.
    
    If `reps` has length ``d``, the result will have dimension of
    ``max(d, A.ndim)``.
    
    If ``A.ndim < d``, `A` is promoted to be d-dimensional by prepending new
    axes. So a shape (3,) array is promoted to (1, 3) for 2-D replication,
    or shape (1, 1, 3) for 3-D replication. If this is not the desired
    behavior, promote `A` to d-dimensions manually before calling this
    function.
    
    If ``A.ndim > d``, `reps` is promoted to `A`.ndim by pre-pending 1's to it.
    Thus for an `A` of shape (2, 3, 4, 5), a `reps` of (2, 2) is treated as
    (1, 1, 2, 2).
    
    Note : Although tile may be used for broadcasting, it is strongly
    recommended to use numpy's broadcasting operations and functions.
    
    Parameters
    ----------
    A : array_like
        The input array.
    reps : array_like
        The number of repetit